In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV_small.json")
df = sqlContext.read.json("s3n://patricks3db/meta_Movies_and_TV.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bought_together: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- buy_after_viewing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- salesRank: struct (nullable = true)
 |    |-- Arts, Crafts & Sewing: long (nullable = true)
 |    |-- Books: long (nullable = true)
 |    |-- Electronics: long (nullable = true)
 |    |-- Health & Personal Care: long (nullable = true)
 |    |-- Home &am

In [3]:
mp = df
mp = mp.drop("salesRank")
mp = mp.drop("related")
mp = mp.drop("categories")
mp.columns

['asin', 'brand', 'imUrl', 'price', 'title']

In [4]:
mp.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[asin: string, brand: string, imUrl: string, price: double, title: string]

In [5]:
newMovieDf = sqlContext.read.json("s3n://patricks3db/mapmidasin/midasin.json")

In [6]:
newMovieDf.take(1)

[Row(asin=u'B000MCWZ4A', mid=10234)]

In [7]:
newMovieDf.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[asin: string, mid: bigint]

In [8]:
mp1 = mp.join(newMovieDf, mp.asin == newMovieDf.asin)
mp1.printSchema()


root
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- mid: long (nullable = true)



In [11]:
mp3 = mp1.map(lambda x: {"mid":x.mid, "imurl":x.imUrl, "asin":x.asin, "title":x.title})

In [12]:
mp3.take(1)

[{'asin': u'B00DNJ984E',
  'imurl': u'http://ecx.images-amazon.com/images/I/51ikh1SPbsL._SY300_.jpg',
  'mid': 187518,
  'title': None}]

In [20]:
def syncToCassandra(d_iter):
        from cqlengine import columns
        from cqlengine.models import Model
        from cqlengine import connection
        from cqlengine.management import sync_table
        CASSANDRA_KEYSPACE = "playground"
        class movieprofile9(Model):
                mid = columns.Integer(primary_key=True)
                asin = columns.Text()
                title = columns.Text()
                imurl = columns.Text()
        connection.setup(['172.31.39.226'], CASSANDRA_KEYSPACE)
        sync_table(movieprofile9)
        for d in d_iter:
                movieprofile9.create(**d)
# Create table if it does not exist. Need to do this before submitting to Spark to avoid collisions
syncToCassandra([])
mp3.foreachPartition(syncToCassandra)